### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/lower_grade_glioma_and_glioblastoma/GSE15309'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# your_code_here

# Check for gene expression data availability
is_gene_available = True  # Assuming the dataset represents gene expression data

# Determine the availability of the variables in the sample characteristics dictionary
# 'lower_grade_glioma_and_glioblastoma'
for key, values in {3: ['cell type: astrocytoma, right parietal occipital lobe, cerebral cortex', 'cell type: Grade II, ER+, PR+, Her2- tumor', 
                         'cell type: glioblastoma, Lt. Temporal, cerebral cortex', 'cell type: glioblastoma, cerebral cortex', 
                         'cell type: GBM']}.items():
    if any('glioma' in value or 'glioblastoma' in value for value in values):
        trait_row = key
        break

# 'age'
for key, values in {2: ['age: 34', 'age: 35 yrs', 'age: 23', 'age: 9', 'age: 10 yrs', 'age: 53 yrs', 'age: 5 yrs', 'age: 66 yrs',
                          'age: 5', 'age: 30 yrs', 'age: 13 yrs', 'age: 65 yrs', 'age: 38 yrs', 'age: 37', 'age: 43 yrs old',
                          'age: 51 yr', 'age: 71 yrs']}.items():
    if any('age' in value for value in values):
        age_row = key
        break

# 'gender'
for key, values in {3: ['sex: male', 'sex: female', 'sex: unknown']}.items():
    if any('sex' in value for value in values):
        gender_row = key
        break

# Define functions to convert variables to appropriate data types
def extract_value(cell):
    """ Helper function to extract the value after the colon in each cell and ensure it's a string """
    cell = str(cell)  # Ensure the cell is treated as a string
    parts = cell.split(':')
    return parts[1].strip() if len(parts) > 1 else None

def convert_trait(value):
    value = extract_value(value)  # Ensure we're working with a string value
    trait_terms = ['glioma', 'glioblastoma']
    if value is not None:
        result = 1 if any(term in value.lower() for term in trait_terms) else 0
    else:
        result = 0
    return result

def convert_age(value):
    value = extract_value(value)  # Process string value first using helper function
    try:
        age_numeric = float(value.split()[0])
        return age_numeric
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    value = extract_value(value)  # Process string value first using helper function
    gender_mapping = {'male': 1, 'female': 0}
    if value is not None:
        return gender_mapping.get(value.lower(), None)
    else:
        return None

# Save cohort information
save_cohort_info('GSE15309', './preprocessed/lower_grade_glioma_and_glioblastoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'lower_grade_glioma_and_glioblastoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/lower_grade_glioma_and_glioblastoma/trait_data/GSE15309.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:

import gzip
import pandas as pd

def check_file_structure(file_path):
    # Print the first 20 lines of the matrix file to analyze its structure
    with gzip.open(file_path, 'rt') as f:
        for i, line in enumerate(f):
            print(line.strip())
            if i >= 19:
                break

# Check the file structure to debug the issue
check_file_structure(matrix_file)

# Modify the get_genetic_data function to handle potential column ID issues
def get_genetic_data(file_path, marker="!series_matrix_table_begin"):
    # Determine the number of rows to skip
    with gzip.open(file_path, 'rt') as file:
        for i, line in enumerate(file):
            if marker in line:
                skip_rows = i + 1  # +1 to skip the marker row itself
                break
        else:
            raise ValueError(f"Marker '{marker}' not found in the file.")

    # Read the genetic data into a dataframe
    genetic_data = pd.read_csv(file_path, compression='gzip', skiprows=skip_rows, comment='!', delimiter='\t',
                               low_memory=False, on_bad_lines='skip')
    
    # Debug: Print the columns to verify the presence of 'ID_REF'
    print("Columns in genetic data:")
    print(genetic_data.columns)
    
    # Ensure the correct column naming scheme is used
    if 'ID_REF' in genetic_data.columns:
        genetic_data = genetic_data.rename(columns={'ID_REF': 'ID'})
    elif 'ID' not in genetic_data.columns:
        raise ValueError("Expected 'ID_REF' or 'ID' column not found.")
    
    genetic_data = genetic_data.dropna()
    genetic_data = genetic_data.astype({'ID': 'str'})
    genetic_data.set_index('ID', inplace=True)

    return genetic_data

# 1. Use the modified get_genetic_data function to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:

# Ensure gene_annotation is defined (carry over from previous steps)
gene_annotation = get_gene_annotation(soft_file)

# 1. Determine the keys for identifiers and gene symbols
identifier_key = 'TAG'
gene_symbol_key = 'GI'

# 2. Get the mapping dataframe between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Print and inspect both gene_data and mapping_df structures
print("Preview of gene_data before mapping:")
print(preview_df(gene_data))

print("Preview of mapping_df:")
print(preview_df(mapping_df))

# 3. Apply the mapping to get the gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("Preview of gene_data after mapping:")
print(preview_df(gene_data))


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/lower_grade_glioma_and_glioblastoma/gene_data/GSE15309.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'lower_grade_glioma_and_glioblastoma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE15309', './preprocessed/lower_grade_glioma_and_glioblastoma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/lower_grade_glioma_and_glioblastoma/GSE15309.csv'
    unbiased_merged_data.to_csv(csv_path)
